# 볼린저밴드
- 이동평균선에 따라 밴드를 그려
- 하단선보다 내려가면 사고 상단선보다 올라가면 판다

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv("../csv/BND.csv", index_col = "Date")
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-04-10,75.070000,75.260002,75.000000,75.239998,51.523693,35000
2007-04-11,75.160004,75.290001,75.029999,75.040001,51.386742,87700
2007-04-12,75.059998,75.080002,74.959999,75.029999,51.379879,78100
2007-04-13,75.040001,75.070000,74.849998,74.910004,51.297688,18000
2007-04-16,74.989998,74.989998,74.940002,74.980003,51.345627,52700


In [16]:
price_df = df.loc[:,["Adj Close"]]
price_df.head()

,Adj Close
Date,
2007-04-10,51.523693
2007-04-11,51.386742
2007-04-12,51.379879
2007-04-13,51.297688
2007-04-16,51.345627


In [17]:
## 이동평균선 파생변수 생성 : 데이터양이 20개의 평균을 구해서 컬럼에 대입
price_df["center"] = price_df["Adj Close"].rolling(20).mean()
price_df.iloc[18:25]

,Adj Close,center
Date,,
2007-05-04,51.679100,NaN
2007-05-07,51.713379,51.556713
2007-05-08,51.720268,51.566542
2007-05-09,51.658497,51.580130
2007-05-10,51.706562,51.596464
2007-05-11,51.637943,51.613477
2007-05-14,51.603649,51.626378


In [18]:
## 상단 밴드 파생변수 생성 (이동 평균선 + 2*표준편차)
price_df["ub"] = price_df["center"] + 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

,Adj Close,center,ub
Date,,,
2007-05-04,51.679100,NaN,NaN
2007-05-07,51.713379,51.556713,51.807910
2007-05-08,51.720268,51.566542,51.827492
2007-05-09,51.658497,51.580130,51.829713
2007-05-10,51.706562,51.596464,51.833301
2007-05-11,51.637943,51.613477,51.804375
2007-05-14,51.603649,51.626378,51.770106


In [19]:
## 하단 밴드 파생변수 생성 (이동 평균선 - 2*표준편차)
price_df["lb"] = price_df["center"] - 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

,Adj Close,center,ub,lb
Date,,,,
2007-05-04,51.679100,NaN,NaN,NaN
2007-05-07,51.713379,51.556713,51.807910,51.305517
2007-05-08,51.720268,51.566542,51.827492,51.305592
2007-05-09,51.658497,51.580130,51.829713,51.330546
2007-05-10,51.706562,51.596464,51.833301,51.359627
2007-05-11,51.637943,51.613477,51.804375,51.422579
2007-05-14,51.603649,51.626378,51.770106,51.482650


In [20]:
##시작 시간
start_time = "2010-01-02"
price_df = price_df.loc[start_time:]
price_df

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,60.611969,60.826958,61.336124,60.317793
2010-01-05,60.789135,60.819153,61.325461,60.312844
2010-01-06,60.766037,60.802128,61.290324,60.313932
2010-01-07,60.719822,60.777802,61.228214,60.327389
2010-01-08,60.781410,60.759625,61.175685,60.343565
...,...,...,...,...
2019-06-18,82.397118,81.737070,82.743959,80.730181
2019-06-19,82.676468,81.836350,82.799510,80.873191
2019-06-20,82.806168,81.934650,82.870640,80.998661


In [21]:
## trade 파생변수 값들은 비어있는 값
price_df["trade"] = ""
price_df.head()

,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,60.611969,60.826958,61.336124,60.317793,
2010-01-05,60.789135,60.819153,61.325461,60.312844,
2010-01-06,60.766037,60.802128,61.290324,60.313932,
2010-01-07,60.719822,60.777802,61.228214,60.327389,
2010-01-08,60.781410,60.759625,61.175685,60.343565,


In [22]:
## 구매내역 데이터를 삽입
## 조건 : [상단 배드보다 종가가 높은 경우]
##         구매 상태 이면
##           매도 trade = ""
##         구매 상태가 아니면
##             아무 행동도 하지 않는다 trade = ""
#                                                   => 다 ""
##      [하단 밴드보다 종가가 낮은 경우]
##         구매 상태이면
##             구매 상태 유지 trade = "buy"
##         구매 생태가 아니면
##              구매 trade = "buy"
#                                                    => 디 buy
##       [상단 배드와 하단 밴드 사이에 종가가 존재하면]
##             구매 상태이면                              이전 행을 shift로 가져와서 구매상태인지 아닌지 if문으로 체크
##                  현 상태 유지 trade = "buy"
##             구매 상태가 아니면
##                  현 상태 유지 trade = ""

for i in price_df.index:
    if price_df.loc[i, "Adj Close"] > price_df.loc[i,"ub"]:
        price_df.loc[i, "trade"] = ""
        
    elif price_df.loc[i, "lb"] > price_df.loc[i,"Adj Close"]:
        price_df.loc[i, "trade"] = "buy"
        
    elif price_df.loc[i, "ub"] >= price_df.loc[i,"Adj Close"] and \
        price_df.loc[i, "Adj Close"] >= price_df.loc[i, "lb"]:
            
        if price_df.shift(1).loc[i, "trade"] == "buy":
            price_df.loc[i, "trade"] = "buy"
        else:
            price_df.loc[i, "trade"] = ""
    

In [23]:
price_df.value_counts("trade")

trade
       1568
buy     816
dtype: int64

In [24]:
# trade열에서 buy인 행만 필터링해서 출력! just 체크 

#price_df[price_df["trade"] == "buy"]
price_df.query("trade == 'buy'")


,Adj Close,center,ub,lb,trade
Date,,,,,
2010-03-25,61.205368,61.569890,61.819734,61.320047,buy
2010-03-26,61.337158,61.560745,61.830838,61.290653,buy
2010-03-29,61.205368,61.541756,61.854636,61.228875,buy
2010-03-30,61.329365,61.526641,61.850257,61.203025,buy
2010-03-31,61.383640,61.513850,61.838911,61.188789,buy
...,...,...,...,...,...
2019-03-01,78.634193,78.905679,79.169995,78.641363,buy
2019-03-04,78.772850,78.907841,79.166744,78.648939,buy
2019-03-05,78.832283,78.916434,79.151029,78.681839,buy


# 2. 손익 계산

In [25]:
rtn = 1.0
price_df["return"] = 1
buy = 0.0
sell = 0.0
for i in price_df.index:
    # 구매하는 경우(구매한 날) (buy가 없다가 생긴날)
    if price_df.loc[i, "trade"] == "buy" and price_df.shift(1).loc[i, "trade"] == "":
        buy = price_df.loc[i, "Adj Close"]
        print("진입일 : ", i, "진입 가격 : ", buy)
    
    # 매도하는 경우
    elif price_df.loc[i, "trade"] == "" and price_df.shift(1).loc[i, "trade"] == "buy":
        sell = price_df.loc[i, "Adj Close"]
        rtn = (sell - buy) / buy + 1 # 손익 계산
        price_df.loc[i, "return"] = rtn
        print("청산일 : ", i, "진입 가격 : ", buy, "청산 가격 : ", sell, "| return : ", round(rtn,4))
        print("----------------------")
        
    # buy, sell 변수 초기화
    if price_df.loc[i, "trade"] == "":
        buy = 0.0
        sell = 0.0

진입일 :  2010-03-25 진입 가격 :  61.205368
청산일 :  2010-05-04 진입 가격 :  61.205368 청산 가격 :  62.186432 | return :  1.016
----------------------
진입일 :  2010-09-10 진입 가격 :  64.517815
청산일 :  2010-09-28 진입 가격 :  64.517815 청산 가격 :  65.29863 | return :  1.0121
----------------------
진입일 :  2010-11-12 진입 가격 :  64.902885
청산일 :  2011-03-16 진입 가격 :  64.902885 청산 가격 :  65.105652 | return :  1.0031
----------------------
진입일 :  2011-06-29 진입 가격 :  66.049957
청산일 :  2011-08-01 진입 가격 :  66.049957 청산 가격 :  67.197464 | return :  1.0174
----------------------
진입일 :  2011-11-30 진입 가격 :  68.584297
청산일 :  2011-12-13 진입 가격 :  68.584297 청산 가격 :  69.032143 | return :  1.0065
----------------------
진입일 :  2012-03-14 진입 가격 :  69.140411
청산일 :  2012-04-10 진입 가격 :  69.140411 청산 가격 :  70.015167 | return :  1.0127
----------------------
진입일 :  2012-08-15 진입 가격 :  71.121567
청산일 :  2013-02-25 진입 가격 :  71.121567 청산 가격 :  71.270378 | return :  1.0021
----------------------
진입일 :  2013-05-10 진입 가격 :  71.452927
청산일 :  2013-09-18 진입

In [26]:
## 누적수익률
acc_rtn = 1.0
for i in price_df.index:
    rtn = price_df.loc[i, "return"]
    acc_rtn = acc_rtn * rtn
    price_df.loc[i, "acc_rtn"] = acc_rtn
    
    print("누적수익률 : ", round(acc_rtn, 4))

누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0
누적수익률 :  1.0

# 3. Class로! : 볼린저밴드 모듈 생성
    - 1. bollinger 폴더를 생성
    - 2. ""__init__.py"파일을 생성
    - 3. 클래스 생성
    - 4. 데이터프레임, 기준이 되는 컬럼 ""__init__"" 함수에서 인자로 받아오는 함수
    - 5. 이동평균선, 상단밴드, 하단밴드, trade="" 생성함수 생성
    - 6. 시작시간은 인자값으로 받아와서 데이터의 개수 필터링
    - 7. 구매 상태를 삽입 함수 생성
    - 8. 손익 계산 함수 생성

In [27]:
import bollinger as big
# df = pd.read_csv("../csv/BND.csv", index_col = "Date")

In [28]:
import FinanceDataReader as fdr
df = fdr.DataReader("AMZN","2009-01-01","2022-09-10")
df

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2009-01-02,2.72,2.57,2.73,2.55,145930000.0,0.0625
2009-01-05,2.70,2.79,2.79,2.65,190230000.0,-0.0074
2009-01-06,2.87,2.73,2.91,2.69,221690000.0,0.0630
2009-01-07,2.81,2.81,2.85,2.77,158860000.0,-0.0209
2009-01-08,2.86,2.75,2.87,2.73,131610000.0,0.0178
...,...,...,...,...,...,...
2022-09-02,127.51,129.78,131.38,126.39,57040000.0,-0.0024
2022-09-06,126.11,127.91,128.53,124.84,43560000.0,-0.0110
2022-09-07,129.48,125.96,129.74,125.43,47750000.0,0.0267


In [29]:
## 클래스 생성
big_df = big.Bollinger(df, 'Close')

#볼린저밴드 컬럼 생성
big_df.add_bollinger()

,Close,Open,High,Low,Volume,Change,center,ub,lb,trade
Date,,,,,,,,,,
2009-01-02,2.72,2.57,2.73,2.55,145930000.0,0.0625,NaN,NaN,NaN,
2009-01-05,2.70,2.79,2.79,2.65,190230000.0,-0.0074,NaN,NaN,NaN,
2009-01-06,2.87,2.73,2.91,2.69,221690000.0,0.0630,NaN,NaN,NaN,
2009-01-07,2.81,2.81,2.85,2.77,158860000.0,-0.0209,NaN,NaN,NaN,
2009-01-08,2.86,2.75,2.87,2.73,131610000.0,0.0178,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...
2022-09-02,127.51,129.78,131.38,126.39,57040000.0,-0.0024,136.2000,148.434679,123.965321,
2022-09-06,126.11,127.91,128.53,124.84,43560000.0,-0.0110,135.5350,148.461305,122.608695,
2022-09-07,129.48,125.96,129.74,125.43,47750000.0,0.0267,135.1175,148.269121,121.965879,


In [30]:
#시작기간 설정
big_df.starttime("2010-01-01")

,Close,Open,High,Low,Volume,Change,center,ub,lb,trade
Date,,,,,,,,,,
2010-01-04,6.70,6.81,6.83,6.66,152010000.0,-0.0045,6.7005,7.073960,6.327040,
2010-01-05,6.73,6.67,6.77,6.59,177130000.0,0.0045,6.6930,7.057192,6.328808,
2010-01-06,6.61,6.73,6.74,6.58,143620000.0,-0.0178,6.6880,7.053951,6.322049,
2010-01-07,6.50,6.60,6.62,6.44,220600000.0,-0.0166,6.6775,7.052726,6.302274,
2010-01-08,6.68,6.53,6.68,6.45,196680000.0,0.0277,6.6830,7.054801,6.311199,
...,...,...,...,...,...,...,...,...,...,...
2022-09-02,127.51,129.78,131.38,126.39,57040000.0,-0.0024,136.2000,148.434679,123.965321,
2022-09-06,126.11,127.91,128.53,124.84,43560000.0,-0.0110,135.5350,148.461305,122.608695,
2022-09-07,129.48,125.96,129.74,125.43,47750000.0,0.0267,135.1175,148.269121,121.965879,


In [31]:
#구매한 상태. 보유한 날 : buy
big_df.buy_state()

trade
       1983
buy    1212
dtype: int64

In [32]:
# 손익계산
big_df.total_rtn()

/Users/kyle/UBION_python/220913/bollinger/__init__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["return"] = 1
/Users/kyle/UBION_python/220913/bollinger/__init__.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.loc[i, "return"] = self.rtn


진입일 :  2010-05-04 00:00:00 진입 가격 :  6.49
청산일 :  2010-08-04 00:00:00 진입 가격 :  6.49 청산 가격 :  6.38 | return :  0.9831
----------------------
진입일 :  2010-11-16 00:00:00 진입 가격 :  7.89
청산일 :  2010-11-24 00:00:00 진입 가격 :  7.89 청산 가격 :  8.86 | return :  1.1229
----------------------
진입일 :  2011-01-21 00:00:00 진입 가격 :  8.87
청산일 :  2011-03-30 00:00:00 진입 가격 :  8.87 청산 가격 :  8.97 | return :  1.0113
----------------------
진입일 :  2011-06-06 00:00:00 진입 가격 :  9.28
청산일 :  2011-06-27 00:00:00 진입 가격 :  9.28 청산 가격 :  10.06 | return :  1.0841
----------------------
진입일 :  2011-08-04 00:00:00 진입 가격 :  10.07
청산일 :  2011-10-14 00:00:00 진입 가격 :  10.07 청산 가격 :  12.34 | return :  1.2254
----------------------
진입일 :  2011-10-26 00:00:00 진입 가격 :  9.92
청산일 :  2012-01-18 00:00:00 진입 가격 :  9.92 청산 가격 :  9.47 | return :  0.9546
----------------------
진입일 :  2012-10-10 00:00:00 진입 가격 :  12.25
청산일 :  2012-11-28 00:00:00 진입 가격 :  12.25 청산 가격 :  12.36 | return :  1.009
----------------------
진입일 :  2013-04-29 00:00:00 진